<a href="https://colab.research.google.com/github/francescopatane96/Computer_aided_drug_discovery_kit/blob/main/ML_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit   #install rdkit library

In [ ]:
!pip install lazypredict  #install lazypredict

In [ ]:
!pip install git+https://github.com/volkamerlab/teachopencadd.git  #install teachopencadd dependecies 

In [ ]:
from pathlib import Path
import seaborn as sns
from warnings import filterwarnings
import time
import lazypredict
from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier

import pandas as pd
import numpy as np
from sklearn import svm, metrics, clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import auc, accuracy_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect

from teachopencadd.utils import seed_everything

# Silence some expected warnings
filterwarnings("ignore")
# Fix seed for reproducible results
SEED = 44
seed_everything(SEED)

In [ ]:
# Read data (Lipinski)
chembl_df = pd.read_csv(
    "IDH_compounds_lipinski.csv",    #read lipinski's descriptors csv
    index_col=0,
)

# Look at head
print("Shape of dataframe : ", chembl_df.shape)
chembl_df.head()


In [ ]:
# Feature for proving and Proving our data \\ NaN finder
def check_missing_values(dataframe):
    
    if dataframe.isnull().sum().sum() > 0:
        m_total = dataframe.isnull().sum().sort_values(ascending=False) 
        total = m_total[m_total > 0]

        m_percent = dataframe.isnull().mean().sort_values(ascending=False) 
        percent = m_percent[m_percent > 0] 

        missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    
        print(f'Total and Percentage of NaN:\n {missing_data}')
    else: 
        print('No NaN found.')
        
        
check_missing_values(dataframe=chembl_df)

In [ ]:
# remove NaN, if present
chembl_df = chembl_df.dropna()

In [ ]:
chembl_df.shape

In [ ]:
# Keep only the columns we want
chembl_df = chembl_df[["molecule_chembl_id", "smiles", "pIC50"]]
chembl_df.head()


In [ ]:
# Add column for activity
chembl_df["active"] = np.zeros(len(chembl_df))

# Mark every molecule as active with an pIC50 of >= 6.3, 0 otherwise
chembl_df.loc[chembl_df[chembl_df.pIC50 >= 6.3].index, "active"] = 1.0

# NBVAL_CHECK_OUTPUT
print("Number of active compounds:", int(chembl_df.active.sum()))
print("Number of inactive compounds:", len(chembl_df) - int(chembl_df.active.sum()))

In [ ]:
chembl_df.head()


install padel (descriptors calculator)

In [ ]:
! wget https://github.com/gromdimon/features/raw/main/padel.sh
! wget https://github.com/gromdimon/features/raw/main/padel.zip

In [ ]:
!unzip padel.zip

In [ ]:
selection = ['smiles', 'molecule_chembl_id']     #select columns we want to retain
act_selected = chembl_df[selection]
act_selected.to_csv('molecule.smi', sep='\t', index=False, header=False )

In [ ]:
! cat molecule.smi | head -5
! cat molecule.smi | wc -l

In [ ]:
!cat padel.sh   #read the script

In [ ]:
!bash padel.sh    #run padel (it reads molecule.smi file that contains canonical form smiles)

In [ ]:
actx = pd.read_csv('descriptors_output.csv') #padel generates a file called 'descriptors_output.csv'
actx                                         

In [ ]:
# Read data (Lipinski)
chembl_df = pd.read_csv(
    "IDH_compounds_lipinski.csv",    #read chembl dataset
    index_col=0,
)

In [ ]:
chembl_df

In [ ]:
#@title optional (categorical classification)

chembl_df = chembl_df[["ro5_fulfilled"]]
chembl_df.head()



In [ ]:
extracted_col = chembl_df["ro5_fulfilled"]
print("column to added from first dataframe to second:")
display(extracted_col)
  
actx = actx.join(extracted_col)
print("Second dataframe after adding column from first dataframe:")
display(actx)

In [ ]:
chembl_df = actx

In [ ]:
def plot_roc_curves_for_models(models, test_x, test_y, save_png=False):
    """
    Helper function to plot customized roc curve.

    Parameters
    ----------
    models: dict
        Dictionary of pretrained machine learning models.
    test_x: list
        Molecular fingerprints for test set.
    test_y: list
        Associated activity labels for test set.
    save_png: bool
        Save image to disk (default = False)

    Returns
    -------
    fig:
        Figure.
    """

    fig, ax = plt.subplots()

    # Below for loop iterates through your models list
    for model in models:
        # Select the model
        ml_model = model["model"]
        # Prediction probability on test set
        test_prob = ml_model.predict_proba(test_x)[:, 1]
        # Prediction class on test set
        test_pred = ml_model.predict(test_x)
        # Compute False postive rate and True positive rate
        fpr, tpr, thresholds = metrics.roc_curve(test_y, test_prob)
        # Calculate Area under the curve to display on the plot
        auc = roc_auc_score(test_y, test_prob)
        # Plot the computed values
        ax.plot(fpr, tpr, label=(f"{model['label']} AUC area = {auc:.2f}"))

    # Custom settings for the plot
    ax.plot([0, 1], [0, 1], "r--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.set_title("Receiver Operating Characteristic")
    ax.legend(loc="lower right")
    # Save plot
    if save_png:
        fig.savefig(f"{DATA}/roc_auc", dpi=300, bbox_inches="tight", transparent=True)
    return fig

In [ ]:
def model_performance(ml_model, test_x, test_y, verbose=True):
    """
    Helper function to calculate model performance

    Parameters
    ----------
    ml_model: sklearn model object
        The machine learning model to train.
    test_x: list
        Molecular fingerprints for test set.
    test_y: list
        Associated activity labels for test set.
    verbose: bool
        Print performance measure (default = True)

    Returns
    -------
    tuple:
        Accuracy, sensitivity, specificity, auc on test set.
    """

    # Prediction probability on test set
    test_prob = ml_model.predict_proba(test_x)[:, 1]

    # Prediction class on test set
    test_pred = ml_model.predict(test_x)

    # Performance of model on test set
    accuracy = accuracy_score(test_y, test_pred)
    sens = recall_score(test_y, test_pred)
    spec = recall_score(test_y, test_pred, pos_label=0)
    auc = roc_auc_score(test_y, test_prob)

    if verbose:
        # Print performance results
        # NBVAL_CHECK_OUTPUT        print(f"Accuracy: {accuracy:.2}")
        print(f"Sensitivity: {sens:.2f}")
        print(f"Specificity: {spec:.2f}")
        print(f"AUC: {auc:.2f}")

    return accuracy, sens, spec, auc

In [ ]:
def model_training_and_validation(ml_model, name, splits, verbose=True):
    """
    Fit a machine learning model on a random train-test split of the data
    and return the performance measures.

    Parameters
    ----------
    ml_model: sklearn model object
        The machine learning model to train.
    name: str
        Name of machine learning algorithm: RF, SVM, ANN
    splits: list
        List of desciptor and label data: train_x, test_x, train_y, test_y.
    verbose: bool
        Print performance info (default = True)

    Returns
    -------
    tuple:
        Accuracy, sensitivity, specificity, auc on test set.

    """
    train_x, test_x, train_y, test_y = splits

    # Fit the model
    ml_model.fit(train_x, train_y)

    # Calculate model performance results
    accuracy, sens, spec, auc = model_performance(ml_model, test_x, test_y, verbose)

    return accuracy, sens, spec, auc

In [ ]:
actx_final = actx.drop('Name', axis=1)
actx_final

In [ ]:
actx_df = actx_final.drop('ro5_fulfilled', axis=1)
actx_df

In [ ]:
chembl_df['ro5_fulfilled']

In [ ]:
# Read data (Lipinski)
chembl_df = pd.read_csv(
    "IDH_compounds_lipinski.csv",
    index_col=0,
)


In [ ]:
X = actx_df               #descriptors
Y = chembl_df.pIC50       #pIC50

In [ ]:
# Spliting data in 80\20 ratio
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=22)

In [ ]:
# Seeing the data that was prepared
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
# Defines and builds the lazyclassifier
reg = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models_train,predictions_train = reg.fit(X_train, X_train, Y_train, Y_train)

In [ ]:
# Performance table of the training set (80% subset)
models_train

In [ ]:
# Checking the study on a test sample
reg = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models_test,predictions_test = reg.fit(X_train,X_test,Y_train,Y_test)

In [ ]:
models_test

In [ ]:
X_train = X_train.astype('int32')
Y_train = Y_train.astype('float64')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor(n_estimators=800, random_state=22)
model.fit(X_train, Y_train)
r2 = model.score(X_test, Y_test)
r2                                 #show explained variance

In [ ]:
# Try data with test sample

Y_pred = model.predict(X_test)
print(Y_pred)

In [ ]:
# Calculate the absolute errors

errors = abs(Y_pred - Y_test)
print('Mean absolute errors:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Calculate percentage of errors
mape = 100 * (errors / Y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

generate model as joblib Object

In [ ]:
import joblib

In [ ]:
joblib.dump(model, "./random_forest.joblib")    #save the model

In [ ]:
loaded_rf = joblib.load("./random_forest.joblib")    #load the model

In [ ]:
loaded_rf.predict(actx_df)              #predict pIC50